In [16]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/2e06e71ef7349a57bc58cc9913ae6bae1f9f8447/mp.py

In [24]:
from glob import glob
import json
import mp
from tqdm import tqdm
from datasets import Dataset
from phonemize import phonemize

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id='mesolitica/TTS', repo_type='dataset', 
    allow_patterns = 'texts/*.json', local_dir = './')

In [4]:
import yaml

config_path = "Configs/config.yml"
config = yaml.safe_load(open(config_path))

In [6]:
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='ms', preserve_punctuation=True,  with_stress=True)

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(config['dataset_params']['tokenizer'])

In [13]:
data = []
for f in glob('texts/*.json'):
    with open(f) as fopen:
        d = json.load(fopen)
    data.extend(d)

In [11]:
len(d)

272785

In [12]:
d[0]

{'original': 'Untuk menangani isu rasuah dan akauntabiliti dalam politik Malaysia, pelbagai pembaharuan dan langkah boleh diambil.',
 'normalized': 'Untuk menangani isu rasuah dan akauntabiliti dalam politik Malaysia , pelbagai pembaharuan dan langkah boleh diambil .'}

In [18]:
len(data)

1205177

In [20]:
phonemize(data[0]['normalized'], global_phonemizer, tokenizer)

{'input_ids': [26,
  3167,
  870,
  2624,
  9,
  21057,
  18,
  228,
  43,
  5,
  258,
  2383,
  9,
  1127,
  97,
  967,
  6],
 'phonemes': ['ˈuntoʔ',
  'mənaŋˈani',
  'ˈisu',
  'rasˈuah',
  'dan',
  'ˌakaʊntˌabilˈiti',
  'dˈalam',
  'polˈiteʔ',
  'məlˈesiə',
  ',',
  'pəlbˈaɡaɪ',
  'pəmbˌaharˈuan',
  'dan',
  'lˈaŋkah',
  'bˈoleh',
  'diˈambel',
  '.']}

In [21]:
def loop(data):
    data, _ = data
    results = []
    for d in tqdm(data):
        r = phonemize(d['normalized'], global_phonemizer, tokenizer)
        r['id'] = '1'
        r['url'] = ''
        r['title'] = ''
        results.append(r)
    return results
        

In [25]:
results = loop((data[:10], 0))

100%|████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 688.88it/s]


In [30]:
results = mp.multiprocessing(data, loop, cores = 20)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

 66%|██████████████████████████████████████████████████████▎                           | 39895/60258 [05:42<02:43, 124.76it/s]/home/husein/.local/lib/python3.10/site-packages/malaya/normalizer/rules.py:424: FutureWarning: Possible nested set at position 3
  if re.findall(_expressions['ic'], word_lower):
 89%|█████████████████████████████████████████████████████████████████████████▎        | 53917/60258 [05:48<00:27, 233.36it/s]/home/husein/.local/lib/python3.10/site-packages/malaya/normalizer/rules.py:424: FutureWarning: Possible nested set at position 3
  if re.findall(_expressions['ic'], word_lower):
 93%|████████████████████████████████████████████████████████████████████████████▎     | 56081/60258 [06:04<00:13, 313.35it/s]/home/husein/.local/lib/python3.10/site-packages/malaya/normalizer/rules.py:424: FutureWarning: Possible nested set at position 3
  if re.findall(_expressions['ic'], word_lower):
 87%|██████████████████████████████████████████████████████████████████████▉         

In [35]:
dataset = Dataset.from_list(results)

In [36]:
dataset.save_to_disk('news')

Saving the dataset (0/1 shards):   0%|          | 0/1205177 [00:00<?, ? examples/s]

In [37]:
dataset[0]

{'input_ids': [26,
  3167,
  870,
  2624,
  9,
  21057,
  18,
  228,
  43,
  5,
  258,
  2383,
  9,
  1127,
  97,
  967,
  6],
 'phonemes': ['ˈuntoʔ',
  'mənaŋˈani',
  'ˈisu',
  'rasˈuah',
  'dan',
  'ˌakaʊntˌabilˈiti',
  'dˈalam',
  'polˈiteʔ',
  'məlˈesiə',
  ',',
  'pəlbˈaɡaɪ',
  'pəmbˌaharˈuan',
  'dan',
  'lˈaŋkah',
  'bˈoleh',
  'diˈambel',
  '.'],
 'id': '1',
 'url': '',
 'title': ''}